# Introduction au Dataframe avec Spark


## 1 Présentation

Pour faciliter le développement de programme, Spark à partir de la version 2 intégre la notion de Dataframe.

Le DataFrame est une couche d'abstraction des RDD, qui présentent les données comme des tables de base de données sans se préoccuper de la taille des données.
Les DataFrames offrent de nombreux avantages:
* Une syntaxe beaucoup plus simple.
* Possibilité d'utiliser SQL directement dans la trame de données.
* La parallélisation des traitements dans une architecture distribuée est gérée par Spark.
    
Si vous avez utilisé R ou même la bibliothèque pandas avec Python, vous connaissez probablement déjà le concept des DataFrames. Même si Spark supporte plusieurs langages il faut savoir que le langage le moins efficace est le python.

Vous êtes prêt, jouons un peu avec les dataframes.

Let's get started!




## 2 Pré-requis

### Démarrer Spark


Pour démarrer votre cluster Spark, si cela n'a pas déjà été fait, vous devrez ouvrir un terminal puis exécuter la commande suivante :

Vérifiez que les containers smaster, sworker1 et sworker2 sont démarrés :

### Connexion à Spark

A partir de ce notebook, vous établirez une connexion avec le cluster Spark en python.
Exécutez la cellule ci-dessous pour vous connecter à votre Cluster Spark  :

In [ ]:
# N'oubliez pas de fermer la connexion à la fin du TP
# spark.stop()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("spark://spark-master:7077").appName("TPDF02").getOrCreate()

Notre objet de connexion 'spark'  dispose d'un ensemble de fonctions pour analyser différentes sources de données : JSON, AVRO, PARQUET, CSV, base de données ...

* `csv(path)`
* `jdbc(url, table, ..., connectionProperties)`
* `json(path)`
* `format(source)`
* `load(path)`
* `orc(path)`
* `parquet(path)`
* `table(tableName)`
* `text(path)`
* `textFile(path)`

On pourra aussi y retrouver des fonctions pour associer des schémas de données à une source de données ou configurer les options de lectures des données :

* `option(key, value)`
* `options(map)`
* `schema(schema)`



## 1 - Les fichiers de données

Pour lire un fichier, il suffit d'utiliser la fonction associée à votre format de fichier:
* csv,txt -> load
* parquet -> parquet
* json -> json 
...

In [ ]:
df_csv = spark.read.load("/dataspark/sales_info.csv", format="csv", sep=",", inferSchema="true", header="true")

Dans les options de lecture du fichier, nous avons demandé à Spark d'utiliser le séparteur ","  et d'inférer le schéma.

Pour consulter le schéma évaluer par Spark utilisez la fonction printSchema()

In [ ]:
df_csv.printSchema()

Pour voir les fonctions disponibles avec les dataframes consultez la documentation Spark :
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

Pour avoir un aperçu des données, utilisez la méthode show()

In [ ]:
df_csv.show()

Vous l'aurez compris Spark utilise un échantillonage des données pour évaluer le schema des données.
Il faut l'avouer, c'est bien plus pratique que les RDD.

Bien entendu, si le schéma ne vous convient pas vous pouvez lui indiquer le schema que vous souhaitez (renommer les noms des colonnes par exemple).

In [ ]:
from pyspark.sql.types import *
# Required for StructField, StringType, IntegerType, etc.


fields = [  StructField("Compagnie", StringType(), True),
            StructField("Personne", StringType(), True),
            StructField("Ventes", FloatType(), False)
            ]
            
csvSchema = StructType(fields)

In [ ]:
df_csv_with_my_schema = spark.read.option("header", "true").option("delimiter", ",").schema(csvSchema).csv("/dataspark/sales_info.csv")

In [ ]:
df_csv_with_my_schema.printSchema()

In [ ]:
df_csv_with_my_schema.show()

Les fichiers csv et texte ne sont pas des formats adaptés pour de grosse volumétrie. 
Une bonne pratique dans les environnements de BIG DATA est de travailler avec des formats binaires pour 2 raisons :
* il consomme moins de stockage.
* il est plus facile de déplacer des données via le réseau.

Dans les écosystèmes HADOOP, les données sont sérialisées sous 2 formes :
* Ligne : Avro, Kryo, Protobuff
* Column : Parquet, Orc

En entreprise, vous trouverez souvent les formats Avro ou parquet pour leur performance.

Pour convertir un fichier en parquet c'est aussi simple que ça :

In [ ]:
df_csv_with_my_schema.write.format("parquet").save("/dataspark/sales_info.parquet")

In [ ]:
!ls /dataspark/sales_info.parquet/

Vous pouvez remarquer qu'au format parquet, notre fichier sales_info.parquet est un répertoire contenant des fichiers part-00000, cela vous rappelle-t'il quelque chose ?
En effet, pour traiter des données parquet sait que les données vont être distribuées et par conséquent applique le principe de partitionnement. Pour un fichier de plusieurs Go, il est possible d'indiquer la taille des partitions parquet.

Bien entendu, la plupart des formats binaires inscrit le schéma des données dans chaque partition.

Vérifions cela :

In [ ]:
Sales_parquet = spark.read.parquet("/dataspark/sales_info.parquet")

In [ ]:
Sales_parquet.printSchema()

In [ ]:
Sales_parquet.show()

On peut facilement convertir d'un format à un autre : 

In [ ]:
Sales_parquet.write.format('json').save('/dataspark/sales.json')

Pour lire ensuite vos données.

In [ ]:
Sales_json = spark.read.json("/dataspark/sales.json")

In [ ]:
Sales_json.show()

# 2 Exploiter les DataFrames

Pour manipuler les DataFrames, nous avons un ensemble de méthodes disponibles :

In [ ]:
df = spark.read.parquet("/dataspark/sales_info.parquet")

In [ ]:
dir(df)

Pour afficher les colonnes :

In [ ]:
df.columns

Travaillons à présent sur les manipulations que nous pouvons faire :

In [ ]:
dfColVentes=df.select('Ventes')

In [ ]:
type(dfColVentes)

Le résultat retourné par le select est un DataFrame

In [ ]:
df.select('Ventes').show()

Pour selectionner les N premières lignes, on trouvera la fonction head :

In [ ]:
# Retourne une d'object RowReturns list of Row objects
df.head(2)

Pour sélectionner multiple colonne :

In [ ]:
df.select(['Compagnie','Ventes']).show()

ou

In [ ]:
df.select(df.Compagnie.alias('test'), df.Ventes).show()

Pour le renommage d'une colonne on peut utiliser la méthode alias :

In [ ]:
df.select(df.Compagnie.alias('col1'), df.Ventes.alias('col2')).show()

Si on veut créer de nouvelles colonnes à partir d'une colonne existante  :

In [ ]:
df.withColumn('VentesAdd',df['Ventes'] * 10).show()

Pour renommer une colonne

In [ ]:
# Simple Rename
df.withColumnRenamed('Compagnie','Company').show()

On peut filtrer sur un critére :

In [ ]:
df.filter(df.Personne=="Sam").show()

Pour les opérations d'agrégation, on utilisera les fonctions du module pyspark.sql.functions lequel regroupe les fonctions max, min, mean ...

In [ ]:
import pyspark.sql.functions as f

df.groupBy('Compagnie').agg(f.max('Ventes').alias('Max Ventes'), f.min('Ventes').alias('Min Ventes'), f.mean('Ventes').alias('Moy_Ventes')).orderBy('Compagnie').show()

Comme pour les bases de données relationnelles, on peut faire des jointures entre des sources de données avec différents formats :

In [ ]:
from pyspark.sql import *
# Création d'un dataframe des localisations des différentes compagnies.
# Le code ci-dessous indique comment créer un dataframe manuellement

Location = Row("Compagnie", "location")
location1 = Location('GOOG', 'US')
location2 = Location('MSFT', 'EUROPE')
location3 = Location('FB', 'ASIA')

locationRows =[location1,location2,location3]
df_location = spark.createDataFrame(locationRows)
type(df_location)

On affiche le contenu de notre nouveau dataframe :

In [ ]:
df_location.show()

On affiche la structure du dataframe :

In [ ]:
df.printSchema()

On réalise la jointure entre le dataframe df listant toutes les ventes des companies et le dataframe df_location précisant la localisation des companies :

In [ ]:
df.join(df_location, df_location.Compagnie == df.Compagnie, "left_outer").show()

### Utilisation du SQL

Pour utiliser des requêtes SQL directement sur un DataFrame, vous devrez l'enregistrer dans une vue temporaire:

In [ ]:
# La méthode createOrReplaceTempView enregistre 
# le DataFrame comme une view temporaire
df.createOrReplaceTempView("sales")

In [ ]:
sql_sales = spark.sql("SELECT * FROM sales")

In [ ]:
sql_sales

In [ ]:
sql_sales.show()

Bien entendu, vous pouvez appliquer des requêtes SQL avec des conditions :

In [ ]:
spark.sql("SELECT * FROM sales WHERE Ventes > 500").show()

Spark SQL supporte un sous ensemble de la norme SQL92. 

Bravo !!!

Vous êtes prêt à faire vos premiers exercices avec Spark.